# Lab-11-3 RNN Long sequence

In [1]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [3]:
# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [4]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

### Making sequence dataset from long sentence(code)

In [6]:
# data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)
    x_data.append([char_dic[c] for c in x_str]) # x str to index
    y_data.append([char_dic[c] for c in y_str]) # x str to index
    
    x_one_hot = [np.eye(dic_size)[x] for x in x_data]
    
    #transform as torch tensor variable
    X = torch.FloatTensor(x_one_hot)
    Y = torch.LongTensor(y_data)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

### Adding FC layer and stacking RNN

In [7]:
# declare RNN + FC
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers,
                               batch_first = True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True)
        
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
net = Net(dic_size, hidden_size, 2)

### Code run through

In [8]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[t] for  t in result])
        else:
            predict_str += char_set[result[-1]]

0 0 odgggd'goo 3.2576003074645996
0 1 dgggd'gooo 3.2576003074645996
0 2 'ggd'goood 3.2576003074645996
0 3 ogg'gdoodg 3.2576003074645996
0 4 d''gddodgg 3.2576003074645996
0 5 ''gododggd 3.2576003074645996
0 6 'gmoodggdg 3.2576003074645996
0 7 o''odggdgo 3.2576003074645996
0 8 ''dgggdgol 3.2576003074645996
0 9 oogggdgold 3.2576003074645996
0 10 ogggdgoldd 3.2576003074645996
0 11 'ggggolddg 3.2576003074645996
0 12 ogggolddgd 3.2576003074645996
0 13 dggolddgdm 3.2576003074645996
0 14 'gogddgdmo 3.2576003074645996
0 15 o'gddgdmdg 3.2576003074645996
0 16 '''ggdmdgo 3.2576003074645996
0 17 o'ggdmdgoo 3.2576003074645996
0 18 'gggmdgooo 3.2576003074645996
0 19 oggmdgooom 3.2576003074645996
0 20 'gmdgooomg 3.2576003074645996
0 21 ''ggooomgg 3.2576003074645996
0 22 'ggooomggd 3.2576003074645996
0 23 ogogomggdd 3.2576003074645996
0 24 doggmggddo 3.2576003074645996
0 25 ogggggddod 3.2576003074645996
0 26 ogmggddodg 3.2576003074645996
0 27 ogggddodgg 3.2576003074645996
0 28 'ggddodggd 3.257600307464

3 34  hi  ho ta 3.0360898971557617
3 35   po ho  p 3.0360898971557617
3 36  tpo tao p 3.0360898971557617
3 37   po ho  p 3.0360898971557617
3 38  ea  po to 3.0360898971557617
3 39  opo ho ea 3.0360898971557617
3 40  tpo ho  p 3.0360898971557617
3 41  aa  po to 3.0360898971557617
3 42  hpo ha ea 3.0360898971557617
3 43  ep  po  a 3.0360898971557617
3 44  hpo ho ta 3.0360898971557617
3 45  to  po to 3.0360898971557617
3 46  ea  po ta 3.0360898971557617
3 47  tpo ho ea 3.0360898971557617
3 48  tu  po ta 3.0360898971557617
3 49  hpo ha  p 3.0360898971557617
3 50  ea  po to 3.0360898971557617
3 51  eo  po ho 3.0360898971557617
3 52  tpo ho  p 3.0360898971557617
3 53  hpo lo ta 3.0360898971557617
3 54  ea  po lo 3.0360898971557617
3 55  ta  pa tp 3.0360898971557617
3 56   podho ea 3.0360898971557617
3 57  tu  po tt 3.0360898971557617
3 58  hpodho ea 3.0360898971557617
3 59  eao po ta 3.0360898971557617
3 60  to  po lo 3.0360898971557617
3 61  hpo po to 3.0360898971557617
3 62  tpo po ta 3.03

6 167   h t  h   2.9904191493988037
6 168 t h h  h   2.9904191493988037
6 169 n u    h   2.9904191493988037
7 0 t to   t t 2.9431347846984863
7 1   t  t t t 2.9431347846984863
7 2 t t t  o t 2.9431347846984863
7 3 t t t  t t 2.9431347846984863
7 4 n t t  t t 2.9431347846984863
7 5 t t td o t 2.9431347846984863
7 6 t t td t t 2.9431347846984863
7 7 o t to t t 2.9431347846984863
7 8 t t od o t 2.9431347846984863
7 9   t t  t t 2.9431347846984863
7 10 o t to t t 2.9431347846984863
7 11 t t td t t 2.9431347846984863
7 12 o t td o t 2.9431347846984863
7 13 n t to t t 2.9431347846984863
7 14 t t t to t 2.9431347846984863
7 15 o td t t t 2.9431347846984863
7 16 t t td o t 2.9431347846984863
7 17 t td t t t 2.9431347846984863
7 18 t t td o t 2.9431347846984863
7 19   t    t t 2.9431347846984863
7 20 t t td t t 2.9431347846984863
7 21 t t t  t t 2.9431347846984863
7 22 t t td o t 2.9431347846984863
7 23 t t td t t 2.9431347846984863
7 24 t t td o t 2.9431347846984863
7 25 t t    t t 2.943134784

10 79  t t dt ml 2.719594717025757
10 80    oto     2.719594717025757
10 81  t t  t e  2.719594717025757
10 82 t  t  t et 2.719594717025757
10 83 ot  l t th 2.719594717025757
10 84 todt u  o  2.719594717025757
10 85 th u  tot  2.719594717025757
10 86  t   t  t  2.719594717025757
10 87  td to   t 2.719594717025757
10 88 t  th t th 2.719594717025757
10 89  t oe  to  2.719594717025757
10 90  t t  t tk 2.719594717025757
10 91 that th t  2.719594717025757
10 92 oe  toe e  2.719594717025757
10 93 e  toth t  2.719594717025757
10 94 t  ol t t  2.719594717025757
10 95  tot  t u  2.719594717025757
10 96 that mto   2.719594717025757
10 97 oe t t   t 2.719594717025757
10 98 th      t  2.719594717025757
10 99  tdt me o  2.719594717025757
10 100  t u  t el 2.719594717025757
10 101  to  t     2.719594717025757
10 102 todt ol t  2.719594717025757
10 103 tk t    ot 2.719594717025757
10 104    o  t th 2.719594717025757
10 105  toeldt oe 2.719594717025757
10 106 thdt dt t  2.719594717025757
10 107    t t

14 0 c to dto e 2.483126163482666
14 1  to dto e  2.483126163482666
14 2 tog th   t 2.483126163482666
14 3 t  to   t  2.483126163482666
14 4   toeg to  2.483126163482666
14 5 tto   toet 2.483126163482666
14 6 the  to to 2.483126163482666
14 7 o g to  o  2.483126163482666
14 8 th thech   2.483126163482666
14 9   thech  t 2.483126163482666
14 10 oto to     2.483126163482666
14 11 theto    t 2.483126163482666
14 12 o toekt t  2.483126163482666
14 13  to    tot 2.483126163482666
14 14 th  o toto 2.483126163482666
14 15 o k  t tot 2.483126163482666
14 16 t   tothe  2.483126163482666
14 17 ct t t t   2.483126163482666
14 18   t the t  2.483126163482666
14 19  toth t  t 2.483126163482666
14 20 tht ettot  2.483126163482666
14 21 ttoe   to  2.483126163482666
14 22 the o to   2.483126163482666
14 23  et  t  kt 2.483126163482666
14 24 e   tot t  2.483126163482666
14 25 ct t e tot 2.483126163482666
14 26 t t e toto 2.483126163482666
14 27  to    to  2.483126163482666
14 28 to    to t 2.48312616348

...

82 23  ip, don't 0.27645841240882874
82 24 ep, don't  0.27645841240882874
82 25 m, don't d 0.27645841240882874
82 26   don't dr 0.27645841240882874
82 27  brn't dru 0.27645841240882874
82 28 ton't drum 0.27645841240882874
82 29  n't drum  0.27645841240882874
82 30 r't drum u 0.27645841240882874
82 31 dt drum up 0.27645841240882874
82 32 t arum up  0.27645841240882874
82 33  arum up p 0.27645841240882874
82 34 toum up pe 0.27645841240882874
82 35  um up peo 0.27645841240882874
82 36  m up peop 0.27645841240882874
82 37 t up peopl 0.27645841240882874
82 38  tp people 0.27645841240882874
82 39 tp people  0.27645841240882874
82 40 t people t 0.27645841240882874
82 41  people to 0.27645841240882874
82 42 teople tog 0.27645841240882874
82 43  nple toge 0.27645841240882874
82 44 rple toget 0.27645841240882874
82 45 rle togeth 0.27645841240882874
82 46  e togethe 0.27645841240882874
82 47 ectogether 0.27645841240882874
82 48 reogether  0.27645841240882874
82 49 to ether t 0.27645841240882874
8

85 137 tong for t 0.26949968934059143
85 138 eng for th 0.26949968934059143
85 139 n' for the 0.26949968934059143
85 140 d for the  0.26949968934059143
85 141 efor the e 0.26949968934059143
85 142 tor the en 0.26949968934059143
85 143  r the end 0.26949968934059143
85 144 n the endl 0.26949968934059143
85 145  tee endle 0.26949968934059143
85 146 toemendles 0.26949968934059143
85 147  erendless 0.26949968934059143
85 148 emendless  0.26949968934059143
85 149 mendless i 0.26949968934059143
85 150 tndless im 0.26949968934059143
85 151 msless imm 0.26949968934059143
85 152 d ess imme 0.26949968934059143
85 153  ess immen 0.26949968934059143
85 154 ecs immens 0.26949968934059143
85 155 ms immensi 0.26949968934059143
85 156 siimmensit 0.26949968934059143
85 157 simmensity 0.26949968934059143
85 158 tmmensity  0.26949968934059143
85 159 mmensity o 0.26949968934059143
85 160  ensity of 0.26949968934059143
85 161  nsity of  0.26949968934059143
85 162 msity of t 0.26949968934059143
85 163 dity 

89 39 tp people  0.2636459767818451
89 40 m people t 0.2636459767818451
89 41 epeople to 0.2636459767818451
89 42 teople tog 0.2636459767818451
89 43 e ple toge 0.2636459767818451
89 44  ple toget 0.2636459767818451
89 45 nle togeth 0.2636459767818451
89 46 ee togethe 0.2636459767818451
89 47 ectogether 0.2636459767818451
89 48  eogether  0.2636459767818451
89 49 to ether t 0.2636459767818451
89 50   ether to 0.2636459767818451
89 51 nether to  0.2636459767818451
89 52 ether to c 0.2636459767818451
89 53  her to co 0.2636459767818451
89 54  em te col 0.2636459767818451
89 55 em te coll 0.2636459767818451
89 56   te colle 0.2636459767818451
89 57  te collec 0.2636459767818451
89 58 to lollect 0.2636459767818451
89 59   lollect  0.2636459767818451
89 60 nbollect w 0.2636459767818451
89 61 tollect wo 0.2636459767818451
89 62 hllect woo 0.2636459767818451
89 63 nlect wood 0.2636459767818451
89 64 eect wood  0.2636459767818451
89 65 ect wood a 0.2636459767818451
89 66  t wood an 0.263645976

93 0 p you want 0.2630067467689514
93 1  tou want  0.2630067467689514
93 2 tou want t 0.2630067467689514
93 3  u want to 0.2630067467689514
93 4 n want to  0.2630067467689514
93 5 iwant to b 0.2630067467689514
93 6 tont to bu 0.2630067467689514
93 7 ont to bui 0.2630067467689514
93 8 nd to buil 0.2630067467689514
93 9 d to build 0.2630067467689514
93 10  do build  0.2630067467689514
93 11 to cuild a 0.2630067467689514
93 12   luild a  0.2630067467689514
93 13 ncuild a s 0.2630067467689514
93 14 tutld a sh 0.2630067467689514
93 15 utld a shi 0.2630067467689514
93 16 ild a ship 0.2630067467689514
93 17 pd a ship, 0.2630067467689514
93 18 e a ship,  0.2630067467689514
93 19  anship, d 0.2630067467689514
93 20 tnship, do 0.2630067467689514
93 21 nship, don 0.2630067467689514
93 22 thip, don' 0.2630067467689514
93 23 iip, don't 0.2630067467689514
93 24 ep, don't  0.2630067467689514
93 25 p, don't d 0.2630067467689514
93 26 e don't dr 0.2630067467689514
93 27  bon't dru 0.2630067467689514
93

96 76 d won't as 0.2591454088687897
96 77  aon't ass 0.2591454088687897
96 78 ton't dssi 0.2591454088687897
96 79  n't assig 0.2591454088687897
96 80  't assign 0.2591454088687897
96 81 dt dssign  0.2591454088687897
96 82 t assign t 0.2591454088687897
96 83 hdssign th 0.2591454088687897
96 84 t sign the 0.2591454088687897
96 85 nkign them 0.2591454088687897
96 86   gn them  0.2591454088687897
96 87  gn them t 0.2591454088687897
96 88 gn them to 0.2591454088687897
96 89   them tos 0.2591454088687897
96 90 dthem tosk 0.2591454088687897
96 91 toem tosks 0.2591454088687897
96 92 hem tosks  0.2591454088687897
96 93 em tosks a 0.2591454088687897
96 94 m tosks an 0.2591454088687897
96 95  tosks and 0.2591454088687897
96 96 tosks and  0.2591454088687897
96 97 hsks and w 0.2591454088687897
96 98 nks and wo 0.2591454088687897
96 99  s and wor 0.2591454088687897
96 100 s and work 0.2591454088687897
96 101  ind work, 0.2591454088687897
96 102 t d dork,  0.2591454088687897
96 103 nd work, b 0.25914